## Importing libraries and downloading data

In [1]:
import os
import pandas as pd
os.system('pip install scholarly')
import scholarly
from nltk import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
# os.chdir('C:\\Users\\Cafral\\Desktop\\kaggle\\CORD-19-research-challenge\\data_v7')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Downloading metadata file

!pip install -q kaggle

f = open("kaggle.json", "w")
f.write('{"username":"pranjalya","key":"fa4deb4f89e2fe282a3e29c166413ecf"}')
f.close()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download allen-institute-for-ai/CORD-19-research-challenge -f metadata.csv
!unzip metadata.csv.zip
!rm -rf metadata.csv.zip

 69% 19.0M/27.6M [00:00<00:00, 97.3MB/s]
100% 27.6M/27.6M [00:00<00:00, 109MB/s] 
Archive:  metadata.csv.zip
  inflating: metadata.csv            


# Load the data

In [0]:
metadata = pd.read_csv('metadata.csv')
metadata.rename(columns={'sha':'paper_id'}, inplace = True)
metadata['paper_id'] = metadata['paper_id'].astype("str")
metadata['title'] = metadata['title'].fillna('nan')
metadata['abstract'] = metadata['abstract'].fillna('nan')

metadata['text'] = metadata['title']+ ' ' + metadata['abstract']
metadata.drop_duplicates(['text'], inplace=True)

In [4]:
metadata.head()

,cord_uid,paper_id,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url,text
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,Airborne rhinovirus detection and effect of ul...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,Discovering human history from stomach bacteri...
2,le0ogx1s,nan,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,A new recruit for the army of the men of death...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,Association of HLA class I with severe acute r...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,A double epidemic model for the SARS propagati...


# Extracting papers which contain ngrams

In [0]:
def find_ngrams(dataframe,columnToSearch,keywords):
    df_w_ngrams = dataframe[dataframe[columnToSearch].str.contains('|'.join(keywords), case=False) == True]
    return df_w_ngrams

ngrams = ['population density','number of people in','number of people per',
                             'highly populated areas','highly populated countries',
                             'densely populated countries','densely populated areas',
                             'high density areas','high density countries'
                             ,'population densities', 'density of population','sparsely populated',
                             'densely populated','density of the population','dense population','populated areas',
                             'densely inhabited','housing density','densely-populated','concentration of people',
                             'population pressure','population studies','populated regions','populous',
                             'high population densities','residential densities','overpopulated']

metadata_ngrams = find_ngrams(metadata,'text',ngrams)
#ngrams_titles.rename(columns={'title':'title_w_ngram'},inplace=True)

# Extracting all sentences from the relevant papers

In [0]:
def metadata_sentences(dataFrame):
    paper_ids = list(dataFrame['paper_id'].unique())
    meta_sent_df = pd.DataFrame(columns=['paper_id','sentence'])
    for paper_id in paper_ids:
        sentence_df = pd.DataFrame(columns=['paper_id','sentence'])
        paper_id_df = dataFrame[dataFrame['paper_id']==paper_id]

        for idx_num,row in paper_id_df.iterrows():
            sentences = [sentence for sentence in sent_tokenize(row.abstract)]
        sentence_df['sentence'] = sentences
        #print(sentence_df['sentence'])
        sentence_df['paper_id'] = paper_id
        meta_sent_df = pd.concat([meta_sent_df,sentence_df])

    meta_sent_df.reset_index(inplace=True)
    meta_sent_df.drop(columns=['index'],inplace=True)
    
    return meta_sent_df

In [0]:
meta_sent_df = metadata_sentences(metadata_ngrams)

# Extracting methodolody, sample size, causal nature, sentences refering to coronavirus, fatality

In [0]:
def extract_features(ngramDf,allSentdataFrame,ngrams):
    
    #ngram sentences
    sentences = find_ngrams(allSentdataFrame,'sentence',ngrams)
    
    # extracting methodology
    methods_list = ['regression','OLS','logistic','time series','model','modelling','simulation','forecast','forecasting']
    methodology = find_ngrams(allSentdataFrame,'sentence',methods_list)

    #extracting sample size
    sample_size_list = ['population size','sample size','number of samples','number of observations','number of subjects']
    sample_size = find_ngrams(allSentdataFrame,'sentence',sample_size_list)

    #extracting nature of correlation
    causal_list =['statistically significant','statistical significance',
                  'correlation','positively correlated','negatively correlated','correlated',
                  'p value','p-value','chi square','chi-square',
                  'confidence interval','CI','odds ratio','OR','coefficient']

    causality_type = find_ngrams(allSentdataFrame,'sentence',causal_list)

    # extracting coronavirus related sentence #can someone check and update this list?
    coronavirus_list = ['severe acute respiratory syndrome','sars-cov','sars-like',
                        'middle east respiratory syndrome','mers-cov','mers-like',
                        'covid-19','sars-cov-2','2019-ncov','sars-2',
                        'sarscov-2','novel coronavirus','corona virus','coronaviruses',
                        'sars','mers','covid19','covid 19']

    coronavirus = find_ngrams(allSentdataFrame,'sentence',coronavirus_list)

    # extracting outcome
    disease_stage_list = ['lethal', 'morbid',"death", "fatality", "mortality","lethal", "lethality", "morbidity"]

    fatality = find_ngrams(allSentdataFrame,'sentence',disease_stage_list)

    df_list = [sentences,methodology,sample_size,causality_type,coronavirus,fatality]
    df_list_name = ['sentences','methodology','sample_size','causality_type','coronavirus','fatality']
    i=0
    for one_df in df_list:
        one_df.rename(columns={'sentence':df_list_name[i]},inplace=True)
        grouped_one_df = one_df.groupby(['paper_id'])[df_list_name[i]].sum()
        ngramDf = pd.merge(ngramDf,grouped_one_df,on='paper_id',how='left')
        i=i+1
    return ngramDf

In [9]:
metadata_ngrams = extract_features(metadata_ngrams,meta_sent_df,ngrams)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
try:
    metadata_ngrams.drop(columns=['cord_uid','source_x','pmcid', 'pubmed_id', 'license', 
                              'Microsoft Academic Paper ID', 'WHO #Covidence',
                              'full_text_file'], inplace=True)
except:
    print("Already removed")

In [0]:
def keywordcounter(sentences, keywords_list):
    '''
    Input : List of sentences, List of keywords
    Returns : Keywords present in sentences, Total count of all keywords present in Input
    '''
    keyword = {}
    sent = " ".join(sentences)
    for pol in keywords_list:
        counter = sent.lower().count(pol)
        if (counter > 0):
            keyword[pol] = counter
    return list(keyword.keys()), sum(keyword.values())

def aggregation(item,keyWordList,RiskFactor):
    '''
    Input : Dataframe of sentences of a paper
    Return : Datframe in Standard Output format
    '''
    dfo = {}
    
    dfo['Risk Factor'] = RiskFactor
    dfo['Title'] = item['title'].iloc[0]
    dfo['Keyword/Ngram'], dfo['No of keyword occurence in Paper'] = keywordcounter(item['text'].tolist(),
                                                                                 keyWordList)
    dfo['Sentences'] = item['sentences'].iloc[0]
    dfo['paper_id'] = item['paper_id'].iloc[0]
    
    dfo['URL'] = item['url'].iloc[0]
    
    dfo['Authors'] = item['authors'].iloc[0]

    try:
        dfo['No of Citations'] = next(scholarly.search_pubs_query(item['title'].iloc[0])).citedby
    except:
        dfo['No of Citations'] = 0
        
    dfo['Correlation'] = item['causality_type'].iloc[0]
    dfo['Design Methodology'] = item['methodology'].iloc[0]
    dfo['Sample Size'] = item['sample_size'].iloc[0]
    dfo['Coronavirus'] = item['coronavirus'].iloc[0]
    dfo['Fatality'] = item['fatality'].iloc[0]
    #dfo['TAXON'] =item['TAXON'].iloc[0]
    
    return dfo

#del df_output
df_output = pd.DataFrame(columns=['Risk Factor', 'Title','Keyword/Ngram', 'No of keyword occurence in Paper',
                                  'paper_id', 'URL',
                                  'Authors','No of Citations', 'Correlation', 
                                  'Design Methodology', 'Sample Size',
                                 'Coronavirus','Fatality'])

grouped = metadata_ngrams.groupby('paper_id')
for key, item in grouped:
    df_output = pd.concat([df_output, pd.DataFrame([aggregation(item,ngrams,'Population Density')])])

df_output = df_output.reset_index()
df_output.to_excel('population_density_metadata.xlsx')

In [13]:
df_output.head()

,index,Risk Factor,Title,Keyword/Ngram,No of keyword occurence in Paper,paper_id,URL,Authors,No of Citations,Correlation,Design Methodology,Sample Size,Coronavirus,Fatality,Sentences
0,0,Population Density,Association of HLA class I with severe acute r...,"[densely populated, populated regions]",2,0104f6ceccf92ae8567a0102f89cbb976969a774,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",118,BACKGROUND: The human leukocyte antigen (HLA) ...,NaN,NaN,During the Taiwan epidemic of severe acute res...,NaN,CONCLUSIONS: Densely populated regions with ge...
1,0,Population Density,Representative Contact Diaries for Modeling th...,[densely populated],1,03b9acce84a4799d4be1152a36abfaf52fb3b523,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,"Fu, Yang-chih; Wang, Da-Wei; Chuang, Jen-Hsiang",50,Recent studies of infectious diseases have att...,We show how such parameters of contact pattern...,NaN,NaN,NaN,To ensure that such diary-based contact patter...
2,0,Population Density,Distribution and Risk Factors of 2009 Pandemic...,[population density],2,043aa68f2c784899f71225e8eb233150760a6a54,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,"Fang, Li-Qun; Wang, Li-Ping; de Vlas, Sake J.;...",45,Data from all reported cases of 2009 pandemic ...,The impact of travel-related risk factors on i...,NaN,NaN,NaN,Lower temperature and lower relative humidity ...
3,0,Population Density,Autocatalytic Model for Covid-19 Progression i...,[population density],1,052bf4fb7deaf593862991af3b118b5f11a9fbe1,https://doi.org/10.1101/2020.04.03.20052985,Anatoly Chernyshev,0,Herewith we present a computational model for ...,Herewith we present a computational model for ...,NaN,Herewith we present a computational model for ...,NaN,The only explicit parameter of the model is th...
4,0,Population Density,Coalescent inference for infectious disease: m...,[population density],1,05513568bb7b58bb7df33023160838ac885a6144,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,"Dearlove, Bethany; Wilson, Daniel J.",38,Genetic analysis of pathogen genomes is a powe...,The aim of this study is to develop robust pop...,NaN,NaN,NaN,We find that differences in genetic diversity ...
